In [51]:
import glob
import pickle
import os
import re
import numpy as np

from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB


In [52]:
def load_data():
    file_paths = [f for f in glob.glob("../data/*.txt")]
    data = {}
    for path in file_paths:
        file = open(path , "rb")
        data[os.path.basename(path)] = pickle.load(file)
        file.close()
    
    file_paths = [f for f in glob.glob("../data/secret_test_set/*.txt")]
    secret_test_data = {}
    for path in file_paths:
        file = open(path , "rb")
        secret_test_data[os.path.basename(path)] = pickle.load(file)
        file.close()
    
    return data, secret_test_data


# this is terrifying code, there ought to be a better way to split these lists into 500 word segments
def segment_data(input_data_dict):
    labels, processed_data = [], []
    for file_path, tag_list in input_data_dict.items():
        label = get_label_from_file_path(file_path)
        string = ''
        for i, tag in enumerate(tag_list):
            string += tag.lemma + ' '
            if i % 500 == 0:
                labels.append(label)
                processed_data.append(string)
                string = ''
        labels.append(label)
        processed_data.append(string)
    return labels, processed_data


def get_label_from_file_path(file_path):
    label = re.search('(?<=-)(.+)(?=-)', file_path)
    if label is None:
        return file_path
    else:
        return label.group()

In [53]:
# Import data

book_data_dict, secret_test_set = load_data()
labels, processed_data = segment_data(book_data_dict)
secret_test_labels, secret_test_processed_data = segment_data(secret_test_set)

In [63]:
# Process data

text_classifier = Pipeline([
    ('vect', CountVectorizer(analyzer='word', ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())
    #('clf', SGDClassifier(loss='hinge', penalty='l2')) # see https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
])

X_train, X_test, y_train, y_test = train_test_split(processed_data, labels, test_size=0.20)


In [64]:
#print(secret_test_set['1892-Zola-LE DEBACLE.txt'][0:100])
#print(secret_test_processed_data[0])
#print(secret_test_labels)
#print(X_test)


In [65]:
text_classifier.fit(X_train, y_train)
    
predicted = text_classifier.predict(X_test)
np.mean(predicted == y_test)
print(metrics.classification_report(y_test, predicted))
    
secret_test_predicted = text_classifier.predict(secret_test_processed_data)
np.mean(secret_test_predicted == secret_test_labels)
print(metrics.classification_report(secret_test_labels, secret_test_predicted))



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                         precision    recall  f1-score   support

           Andre Breton       0.00      0.00      0.00        12
             Andre Gide       0.00      0.00      0.00        62
               Bernanos       0.00      0.00      0.00        20
                 Celine       0.00      0.00      0.00       201
          Chateaubriand       0.00      0.00      0.00        16
        Claude Farrère       0.00      0.00      0.00        23
                Colette       0.00      0.00      0.00        91
       Françoise Sagan       0.00      0.00      0.00        10
              Giraudoux       0.00      0.00      0.00        31
        Gobineau Joseph       0.00      0.00      0.00        46
               Huysmans       0.00      0.00      0.00        43
             Jean Giono       0.00      0.00      0.00        14
       Jean Paul Sartre       0.00      0.00      0.00        51
         Joseph Malegue       0.00      0.00      0.00       146
           Jules Renard 

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
